In [1]:
import numpy as np

import dolfinx

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log,default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, Expression )
from dolfinx.fem.petsc import NonlinearProblem,LinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import XDMFFile
import ufl
from ufl import (TestFunctions, TrialFunction, Identity, grad, det, div, dev, inv, tr, sqrt, conditional , gt, dx, inner, derivative, dot, ln, split,TestFunction,indices,as_tensor)
from basix.ufl import element, mixed_element, quadrature_element
from datetime import datetime
from dolfinx.plot import vtk_mesh
from dolfinx.fem.petsc import  assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc,create_matrix, create_vector
import basix
import pyvista
pyvista.set_jupyter_backend('client')
## Define temporal parameters

In [2]:
# Geometric parameters
geom = {"longside" : 100.0,     # mm
        "side" : 10.0,      # mm
        "num_elements" : 3,    # size of a cell
        }


# Mechanicals parameters
mech = {"E" : 200e3,    # MPa
        "nu" : 0.3,     #       
        "sig0" : 100.,  # MPa
        "H" : 80e3, # MPa
        "r" : 2.0,
        "Y_s" : 250.0 #mpa
        
        }


# Study parameters
stud = {"deg u" : 4,    # Interpolation of u
        "deg sig" : 2,  # Interpolation of sig, eps, p
        "N incr" : 100,  # Number of load steps
        "Max Disp" : 1 # Maximal displacement
        }


In [3]:
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0],[geom["longside"],geom["side"],geom["side"]]],[geom["num_elements"]*10,geom["num_elements"],geom["num_elements"]])

In [4]:
E = Constant(domain, mech["E"])
nu = Constant(domain, mech["nu"])
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2.0 / (1 + nu)
sig0 = Constant(domain, mech["sig0"])  # yield strength
H_0 = Constant(domain,mech["H"])  # hardening modulus
r = Constant(domain,mech["r"])  # hardening modulus
Y_s = Constant(domain,mech["Y_s"])  # hardening modulus

In [5]:
deg_u = stud["deg u"]
deg_stress = stud["deg sig"]
Ve = element(
    "Lagrange", domain.basix_cell(), deg_u, shape=(3,)
)  # 2 degrees  of freedom
V = functionspace(domain, Ve)

Ve_scal = element(
    "Lagrange", domain.basix_cell(), deg_u
)
V_scal = functionspace(domain, Ve_scal)

We = quadrature_element(domain.basix_cell(), value_shape=(6,), degree=deg_stress,scheme='default')
W = functionspace(domain, We)

W_scal_e = quadrature_element(domain.basix_cell(), degree=deg_stress,scheme='default')
W_scal = functionspace(domain, W_scal_e)

In [6]:
T = Function(W,name = "Stress")
E_p = Function(W, name="Total_Plastic_Strain")
e_p= Function(W_scal, name="Equivalent_Plastic_Strain")
u = Function(V, name="Total_displacement")
du = Function(V, name="Trial_displacement")

dp = Function(W_scal,name="Delta_plasticity")

Y = Function(W_scal,name="Isotropic Hardening")
A = Function(W_scal,name="Kinematic Hardening")

Y.interpolate(lambda x: np.full_like(x[0],mech["sig0"]))


v = TestFunction(V) #Function we are testing with
du_ = TrialFunction(V) #Function we are solving for

e_p_ = TrialFunction(W_scal)
e_pv = TestFunction(W_scal)


dx = ufl.Measure("dx",domain=domain,  metadata={"quadrature_degree": 2, "quadrature_scheme": "default"} )
n = ufl.FacetNormal(domain)

In [8]:

def tensor_to_vector(X): 
    ''' 
    Take a 3x3 tensor and return a vector of size 4 in 2D
    '''
    return ufl.as_vector([X[0, 0], X[1, 1], X[2, 2], X[0, 1], X[0, 2], X[1, 2]])

# Testing Interpolating into Quadrature with an expression

In [9]:
u.sub(0).interpolate(lambda x: x[0]*2)

In [10]:
Strain = ufl.sym(grad(u))
Strain_vec = tensor_to_vector(Strain)

Strain_exp = Expression(Strain_vec,W.element.interpolation_points())
E_p.interpolate(Strain_exp) 


In [11]:
E_p.x.array[:]

array([ 2.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        9.32587341e-16, -1.27336467e-15,  0.00000000e+00])

# Forming a vector

For forming a vector we need a test function included in the measure.

In [12]:
value = (E_p[0]-Strain_vec[0])*e_pv*dx # We need a trial function for it to work ( in the same one)

Vec = assemble_vector( dolfinx.fem.form((value)))

In [13]:
Vec.norm()

0.0

# Interpolating Qudature into a Function

In [14]:
quadrature_points, weights = basix.make_quadrature(domain.basix_cell(), 2)


In [15]:
W.element.interpolation_points().shape

(4, 3)

In [16]:
E_00 = Function(V_scal)

In [17]:
quadrature_points

array([[0.5854102, 0.1381966, 0.1381966],
       [0.1381966, 0.5854102, 0.1381966],
       [0.1381966, 0.1381966, 0.5854102],
       [0.1381966, 0.1381966, 0.1381966]])

In [20]:
Quad_Exp=  Expression(E_p[0],V_scal.element.interpolation_points())
E_00.interpolate(Quad_Exp)

ValueError: Mismatch of tabulation points and element points.